In [65]:
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import tqdm
from tqdm.auto import tqdm;

In [66]:
# Make a device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

In [82]:
device

'cuda'

In [67]:
#GPUS
n_gpus = torch.cuda.device_count()
print(f"Number of gpus: {n_gpus}")

Number of gpus: 1


In [68]:
class ConvBlock (nn.Module):
    def __init__(self,
                in_channels: int,
                out_channels: int,
                 **kwargs): # this means that the key arguments are arbitrary
        super().__init__()

        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                              **kwargs,
                            device=device)
        self.batchnorm = nn.BatchNorm2d(num_features=out_channels) # to improve performance
    
    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.relu(x)
        return x

In [69]:
class L2NormLayer(nn.Module):
    def __init__(self,dim=1):
        super().__init__()
        self.dim=dim
    
    def forward(self,x):
        return nn.functional.normalize(x,p=2,dim=self.dim)

In [70]:
class InceptionBlock (nn.Module):
        def __init__(self,
                in_channels: int,
                out_1x1: int,

                red_3x3: int,
                out_3x3: int,

                red_5x5: int,
                out_5x5: int,

                out_1x1pool: int,
                
                l2_mode:bool=False):
        
                super().__init__()

                self.branch1 = ConvBlock(in_channels=in_channels,
                                        out_channels=out_1x1,
                                        kernel_size=1)
                
                self.branch2 = nn.Sequential(
                        ConvBlock(in_channels=in_channels,
                                out_channels=red_3x3,
                                kernel_size=1),
                        ConvBlock(in_channels=red_3x3,
                                out_channels=out_3x3,
                                kernel_size=3,
                                padding=1) # ojo (btw no ponemos el S bc por defecto es 1)
                )
                
                self.branch3 = nn.Sequential(
                        ConvBlock(in_channels=in_channels,
                                out_channels=red_5x5,
                                kernel_size=1),
                        ConvBlock(in_channels=red_5x5,
                                out_channels=out_5x5,
                                kernel_size=5,
                                padding=2) # ojo (btw no ponemos el S bc por defecto es 1)
                )

                if l2_mode:
                        self.branch4=nn.Sequential(
                                L2NormLayer(dim=1),
                                ConvBlock(in_channels=in_channels,
                                out_channels=out_1x1pool,
                                kernel_size=1)
                        )
                else:
                        self.branch4 = nn.Sequential(
                                nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
                                ConvBlock(in_channels=in_channels,
                                        out_channels=out_1x1pool,
                                        kernel_size=1)
                        )

        def forward(self,x):
        # N x filters x 28 x 28 → 0th x 1st x 2nd x 3rd dimension (we use 1)
                return torch.cat([self.branch1(x),self.branch2(x),self.branch3(x),self.branch4(x)],1)

In [71]:
class InceptionBlock_2(nn.Module):
    def __init__(self, 
                in_channels:int, 
                red_3x3:int,
                out_3x3:int) -> None:
        super().__init__()
        self.branch=nn.Sequential(
            ConvBlock(in_channels=in_channels,
                    out_channels=red_3x3,
                    kernel_size=1),
            ConvBlock(in_channels=red_3x3,
                    out_channels=out_3x3,
                    kernel_size=3)
        )
    def forward(self,x):
        return self.branch(x)

In [202]:
class InceptionBlock_m_3x3 (nn.Module):
        def __init__(self,
                in_channels: int,

                red_3x3: int,
                out_3x3: int,

                red_5x5: int,
                out_5x5: int):
        
                super().__init__()

                self.branch2 = nn.Sequential(
                        ConvBlock(in_channels=in_channels,
                                out_channels=red_3x3,
                                kernel_size=1),
                        ConvBlock(in_channels=red_3x3,
                                out_channels=out_3x3,
                                kernel_size=3,
                                padding=1,
                                stride=2) # ojo (btw no ponemos el S bc por defecto es 1)
                )
                
                self.branch3 = nn.Sequential(
                        ConvBlock(in_channels=in_channels,
                                out_channels=red_5x5,
                                kernel_size=1),
                        ConvBlock(in_channels=red_5x5,
                                out_channels=out_5x5,
                                kernel_size=5,
                                padding=2,
                                stride=2) # ojo (btw no ponemos el S bc por defecto es 1)
                )

                self.branch4 = nn.Sequential(
                        nn.MaxPool2d(kernel_size=3,stride=2,padding=1), #TODO: VERIFICAR LO DEL PADDING CON JAVIER
                )

        def forward(self,x):
        # N x filters x 28 x 28 → 0th x 1st x 2nd x 3rd dimension (we use 1)
                return torch.cat([self.branch2(x),self.branch3(x),self.branch4(x)],1)

In [218]:
class NN2 (nn.Module):
    def __init__(self,
                in_channels = 3):
        super().__init__()

        self.conv1 = ConvBlock(in_channels=in_channels,
                            out_channels=64,
                            kernel_size=7,
                            stride=2,
                            padding=3)
        
        self.inception2 = InceptionBlock_2(in_channels=64, #la cantidad de canales que entran y salen es la misma. ¿esto es correcto?
                                        red_3x3=64,
                                        out_3x3=192)
        
        self.inception3a = InceptionBlock( in_channels= 192, out_1x1= 64, red_3x3= 96, out_3x3= 128, red_5x5= 16,out_5x5= 32, out_1x1pool= 32)#ok
        self.inception3b = InceptionBlock( in_channels= 256, out_1x1= 64, red_3x3= 96, out_3x3= 128, red_5x5= 32,out_5x5= 64, out_1x1pool= 64,l2_mode=True)
        #modificar desactivando branch 1 y 4
        self.inception3c = InceptionBlock_m_3x3( in_channels= 320, red_3x3= 128, out_3x3= 256, red_5x5= 32,out_5x5= 64)
        
        self.inception4a = InceptionBlock( in_channels= 640, out_1x1= 256, red_3x3= 96, out_3x3= 192, red_5x5= 32,out_5x5= 64, out_1x1pool= 128,l2_mode=True)
        self.inception4b = InceptionBlock( in_channels= 640, out_1x1= 224, red_3x3= 112, out_3x3= 224, red_5x5= 32,out_5x5= 64, out_1x1pool= 128,l2_mode=True)
        self.inception4c = InceptionBlock( in_channels= 640, out_1x1= 192, red_3x3= 128, out_3x3= 256, red_5x5= 32,out_5x5= 64, out_1x1pool= 128,l2_mode=True)
        self.inception4d = InceptionBlock( in_channels= 640, out_1x1= 160, red_3x3= 144, out_3x3= 288, red_5x5= 32,out_5x5= 64, out_1x1pool= 128,l2_mode=True)
        #modificar desactivando branch 1 y 4
        self.inception4e = InceptionBlock_m_3x3( in_channels= 640, red_3x3= 160, out_3x3= 256, red_5x5= 64,out_5x5= 128)
        
        self.inception5a = InceptionBlock( in_channels= 1024, out_1x1= 384, red_3x3= 192, out_3x3= 384, red_5x5= 48,out_5x5= 128, out_1x1pool= 128,l2_mode=True)
        self.inception5b = InceptionBlock( in_channels= 1024, out_1x1= 384, red_3x3= 192, out_3x3= 384, red_5x5= 48,out_5x5= 128, out_1x1pool= 128)#ok
        
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1) #por que se aplico un max pool?

        self.avgpool = nn.AvgPool2d(kernel_size=7) # stride? padding?
        
        self.FC = nn.Linear(1024,128)

        self.norm=nn.BatchNorm2d(num_features=64)


    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool(x)

        x=self.norm(x)

        x = self.inception2(x)
        x = self.maxpool(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.inception3c(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)

        x = self.inception5a(x)
        x = self.inception5b(x)
        #print('Despues del inception 5b',x.shape)
        x = self.avgpool(x)
        #print('Despues del avg pool',x.shape)

        x=x.view(x.shape[0],-1)

        x = self.FC(x)
        #print('Despues de fully connected layer',x.shape)
        x = nn.functional.normalize(x,p=2,dim=1)

        return x

In [214]:
torch.manual_seed(42)
model_1 = NN2(in_channels=3).to(device)

In [215]:
#Sample de prueba

sample=torch.randn(size=(1,3,224,224)).to(device)

In [216]:
sample.shape

torch.Size([1, 3, 224, 224])

In [217]:
model_1.eval()
with torch.inference_mode():
    model_1(sample)

Despues del inception 5b torch.Size([1, 1024, 7, 7])
Despues del avg pool torch.Size([1, 1024, 1, 1])
Despues de fully connected layer torch.Size([1, 128])
